<a href="https://colab.research.google.com/github/josephgills/DataSociety/blob/main/WeatherGrab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [30]:
import re
import requests
from datetime import datetime, timedelta

def parse_lat_lon(input_string):
    # Regular expression to extract latitude and longitude
    pattern = r'(\d+\.\d+)°?\s*([NS]),?\s*(\d+\.\d+)°?\s*([EW])'
    match = re.match(pattern, input_string)

    if match:
        # Extract latitude and longitude values
        lat_deg = float(match.group(1))
        lat_dir = 1 if match.group(2) == 'N' else -1
        lon_deg = float(match.group(3))
        lon_dir = 1 if match.group(4) == 'E' else -1

        # Convert degrees to decimal format
        latitude = lat_deg * lat_dir
        longitude = lon_deg * lon_dir

        return latitude, longitude
    else:
        try:
            # Split the input string by comma and whitespace
            parts = input_string.split(',')
            latitude = float(parts[0])
            longitude = float(parts[1])
            return latitude, longitude
        except Exception as e:
            print("Invalid input format.")
            return None, None

def get_forecast_period_by_name(period_name, forecast_data):
    periods = forecast_data['properties']['periods']
    for period in periods:
        if period['name'] == period_name:
            return period
    return None

def get_forecast_temperature(lat_lon, period_name):
    forecast_url = f"https://api.weather.gov/points/{lat_lon[0]},{lat_lon[1]}"

    # Fetch metadata
    response = requests.get(forecast_url)
    metadata = response.json()

    # Get forecast URL from metadata
    forecast_url = metadata['properties']['forecast']

    # Fetch forecast data
    response = requests.get(forecast_url)
    forecast_data = response.json()

    # Get the forecast period by name
    period = get_forecast_period_by_name(period_name, forecast_data)

    if period:
        return period['temperature'], period['temperatureUnit']
    else:
        return None, None

if __name__ == "__main__":
    # User input
    input_string = input("Enter a latitude and longitude (in decimal format) or in the format '38.2527° N, 85.7585° W': ")
    period_name = input("Select the forecast period name (e.g., 'Wednesday', 'Wednesday Night', etc.): ")

    latitude, longitude = parse_lat_lon(input_string)

    if latitude is not None and longitude is not None:
        temperature, unit = get_forecast_temperature((latitude, longitude), period_name)
        if temperature and unit:
            print(f"Temperature for {period_name} at {latitude}° N, {longitude}° W: {temperature}°{unit}")
        else:
            print(f"Temperature for {period_name} at {latitude}° N, {longitude}° W not available.")
    else:
        print("Invalid input format.")

Enter a latitude and longitude (in decimal format) or in the format '38.2527° N, 85.7585° W': 38.2527° N, 85.7585° W
Select the forecast period name (e.g., 'Wednesday', 'Wednesday Night', etc.): Wednesday
Temperature for Wednesday at 38.2527° N, -85.7585° W: 58°F
